In [2]:
import os
import json
from pathlib import Path
import pandas as pd # type: ignore
import pymupdf # type: ignore
from tqdm.notebook import tqdm # type: ignore

In [3]:
with open("cpv_divisions.json", "r", encoding="utf-8") as file:
    cpv_mapping = json.load(file)

In [4]:
current_dir = Path(os.getcwd())
pdf_paths = list(current_dir.rglob("*.pdf"))

data = {
    "file": [],
    "text": [],
    "page": [],
    "division_number": [],
    "division_title": [],
    "language": [],    
}

for p in tqdm(pdf_paths):
    try:
        cft_root = p.parts.index("docs") - 1
        file = "/".join(p.parts[cft_root:])
        
        doc = pymupdf.open(p)

        for idx, page in enumerate(doc):
            data["file"].append(file)
    
            data["text"].append(page.get_text())
        
            data["page"].append(idx+1)
            
            division_number = int(p.parts[p.parts.index("intelliprocure_data") + 1])
            data["division_number"].append(division_number)
        
            division_title = cpv_mapping.get(str(division_number), None)
            data["division_title"].append(division_title)
        
            lang = p.parts[p.parts.index("intelliprocure_data") + 2]
            data["language"].append(lang)

    except:
        print("Could not open file", file)

df = pd.DataFrame(data)
df.head()

  0%|          | 0/1505 [00:00<?, ?it/s]

MuPDF error: library error: zlib error: incorrect header check

MuPDF error: format error: corrupt object stream (2956 0 R)

MuPDF error: unsupported error: cannot create appearance stream for LinkId annotations

MuPDF error: unsupported error: cannot create appearance stream for LinkId annotations

MuPDF error: unsupported error: cannot create appearance stream for LinkId annotations

MuPDF error: unsupported error: cannot create appearance stream for LinkId annotations

MuPDF error: format error: object (482 0 R) was not found in its object stream

MuPDF error: format error: object (162 0 R) was not found in its object stream

MuPDF error: format error: object (7 0 R) was not found in its object stream

Could not open file 228457/docs/annexe_2_declaration_egalite_hommes_femmes_v2019.pdf
Could not open file 228457/docs/annexe_3_declaration_devel_durable_v2019.pdf
Could not open file 228457/docs/annexe_4_questionnaire_contrib_composante_sociale_dd_services_juin2019.pdf
Could not open f

,file,text,page,division_number,division_title,language
0,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Ausschreibung und Angebot Nr. 325\nProjekt:\n1...,1,92,"Recreational, cultural and sporting services",DE
1,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 2\n14.07.2023\nProjekt: 1156.001\nGlaru...,2,92,"Recreational, cultural and sporting services",DE
2,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 3\n14.07.2023\nProjekt: 1156.001\nGlaru...,3,92,"Recreational, cultural and sporting services",DE
3,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 4\n14.07.2023\nProjekt: 1156.001\nGlaru...,4,92,"Recreational, cultural and sporting services",DE
4,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 5\n14.07.2023\nProjekt: 1156.001\nGlaru...,5,92,"Recreational, cultural and sporting services",DE


In [5]:
print(df.sample().text.item())

EMBASSY OF SWITZERLAND, RENOVATION 
55-17109-00 
TN10/3805.018 WASHINGTON, KA; GESAMTASNIERUNG 
29 APRIL 2019 
 
BID SET 
 
 
 
MODULAR INDOOR CENTRAL-STATION AIR-HANDLING 
UNITS 
© 2019 DLR Group 
237313-9 
a. 
Seamless copper with die-formed holes to provide a parallel surface to the coil 
tube for strong brazing joints.  Coil is supplied with 1/8" brass female pipe thread 
(FPT) vents and drains.  All circuiting is designed to gravity-drain. 
4. 
Connections 
a. 
Red Brass Schedule 40 male pipe thread (MPT) to prevent dielectric reaction 
between dissimilar metals.  
5. 
Casing 
a. 
Minimum 16 gauge 304 stainless steel with 1-1/2" die-formed flanges to permit 
easy stacking and mounting.  Intermediate tube supports are supplied on coils over 
44" fin length with additional supports every 42" multiple thereafter.  
6. 
Testing and Performance 
a. 
All coil assemblies are leak tested under water at 500 PSIG.  Standard construction 
is suitable for 250 PSIG operating pressure up to 300

In [13]:
df

,file,text,page,division_number,division_title,language
0,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Ausschreibung und Angebot Nr. 325\nProjekt:\n1...,1,92,"Recreational, cultural and sporting services",DE
1,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 2\n14.07.2023\nProjekt: 1156.001\nGlaru...,2,92,"Recreational, cultural and sporting services",DE
2,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 3\n14.07.2023\nProjekt: 1156.001\nGlaru...,3,92,"Recreational, cultural and sporting services",DE
3,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 4\n14.07.2023\nProjekt: 1156.001\nGlaru...,4,92,"Recreational, cultural and sporting services",DE
4,261790/docs/04_lv_auskleidung_mit_kunststofffo...,Seite: 5\n14.07.2023\nProjekt: 1156.001\nGlaru...,5,92,"Recreational, cultural and sporting services",DE
...,...,...,...,...,...,...
20317,253079/docs/sam_kantuntani_rfp_final_230301_00...,\n \n \n9/11 \n10. Components \nThe integral ...,69,22,Printed matter and related products,EN
20318,253079/docs/sam_kantuntani_rfp_final_230301_00...,\n \n \n10/11 \nIf provision of the contractu...,70,22,Printed matter and related products,EN
20319,253079/docs/sam_kantuntani_rfp_final_230301_00...,\n \n \n11/11 \nThe Contracting Entity: \nFo...,71,22,Printed matter and related products,EN
20320,253079/docs/sam_kantuntani_rfp_final_230301_00...,SECO / Swiss Accompanying Measures (SAM): Requ...,72,22,Printed matter and related products,EN


In [27]:
# Total number of Files
print("Total number of Files",len(df['file'].unique()))

# Total number of files in EN, FR, DE, IT
language_counts = df.groupby('language')['file'].nunique()
print("\nTotal number of files by language:")
print(language_counts)

# Average number of pages per file in EN, DE
average_pages = (
    df[df['language'].isin(['EN', 'DE'])]
    .groupby('language')
    .agg(total_pages=('page', 'count'), total_files=('file', 'nunique'))
)
# Add average pages per file column
average_pages['avg_pages_per_file'] = average_pages['total_pages'] / average_pages['total_files']

print("Average number of pages per file for EN and DE:")
print(average_pages[['avg_pages_per_file']])

Total number of Files 1499

Total number of files by language:
language
DE    454
EN    109
FR    450
IT    486
Name: file, dtype: int64
Average number of pages per file for EN and DE:
          avg_pages_per_file
language                    
DE                  9.121145
EN                 40.577982


In [28]:
from nltk.tokenize import word_tokenize


def count_tokens(text):
    return len(word_tokenize(text))

# Add a 'tokens' column with the token count for each page
tqdm.pandas()  # Enable progress bar for Pandas
df['tokens'] = df['text'].progress_apply(count_tokens)

# Calculate the average number of tokens per page
average_tokens_per_page = df['tokens'].mean()

# Display the result
print(f"Average number of tokens per page: {average_tokens_per_page:.2f}")

  0%|          | 0/20322 [00:00<?, ?it/s]

Average number of tokens per page: 330.45
